In [46]:
import tensorflow as tf
from numpy.random import RandomState

batch_size = 8

theta1 = tf.Variable(tf.random_normal((2, 3), stddev = 1, seed = 1))#随机初始参数，用正太分布，seed = 1保证每次初始运行得到一样的结果
theta2 = tf.Variable(tf.random_normal((3, 1), stddev = 1, seed = 1))

x = tf.placeholder(tf.float32, shape = (None,2), name = "x-input")  #导入样本
y_ = tf.placeholder(tf.float32, shape = (None,1),name="y-input") #y_表示样本label实际值
global_step = tf.Variable(0)
#前向传播得到输出结果
a = tf.matmul(x, theta1)
y = tf.matmul(a, theta2)
#参数优化
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0))+(1-y_)*tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
# learn_rate = 0.001
learn_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.96, staircase = True) #学习率指数衰减
train_step = tf.train.AdamOptimizer(learn_rate).minimize(cross_entropy, global_step = global_step)
#生成模拟数据集
rdm = RandomState(1)  #1为随机种子，保证每次生成的随机数都相同
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[int(x1+x2<1)] for (x1,x2) in X]  #小于1的样本被认为是正样本，给所有样本加标签 ###列表推导式###


with tf.Session() as sess:
    init_op = tf.global_variables_initializer() #变量初始化
    sess.run(init_op)
    print(sess.run(theta1))
    print(sess.run(theta2))

    steps = 5000
    for i in range(steps):
        start = (i*batch_size)%dataset_size
        end = min(start+batch_size, dataset_size)
        sess.run(train_step, feed_dict={x:X[start:end],y_:Y[start:end]})
        if i%1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x:X,y_:Y})
            print('after %d training steps, cross entropy on all datais %g' % (i, total_cross_entropy))

    print(sess.run(theta1))
    print(sess.run(theta2))



[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
after 0 training steps, cross entropy on all datais 1.60187
after 1000 training steps, cross entropy on all datais 0.608305
after 2000 training steps, cross entropy on all datais 0.608156
after 3000 training steps, cross entropy on all datais 0.60812
after 4000 training steps, cross entropy on all datais 0.608093
[[ 1.3741788e-04  1.7485559e-02  1.5001397e+00]
 [-2.9927515e-04 -3.7934620e-02  5.4605708e-02]]
[[ 7.1550380e-07]
 [ 6.1507657e-05]
 [-1.0732832e+00]]
